<a href="https://colab.research.google.com/github/cerisakatchmart/PicklistCode2024/blob/main/SVR24CrescendoPicklist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Authentification

In [ ]:
# import libs needed to process data
import numpy as np
import pandas as pd
from scipy import stats as st

import matplotlib.pyplot as plt
import seaborn as sns
# imports libs needed to access data from google sheets
import google.colab as colab
import gspread
import google.auth as gauth

# sets up authentification
colab.auth.authenticate_user()
creds, _ = gauth.default()
gc = gspread.authorize(creds)

# setting up data table display
from google.colab import data_table
data_table.enable_dataframe_formatter()

In [ ]:
# main scouting response sheet
sheet_name = "Crescendo SVR 2024"
source_pg = "SVR"

sheet = gc.open(sheet_name)
source = sheet.worksheet(source_pg)
df = pd.DataFrame(source.get_all_records())

# picklist sheet
pl_sheet_name = "SVR 2024 Picklist"
pl_pg = "Picklist"

picklist_fin = gc.open(pl_sheet_name)
picklist_pg = picklist_fin.worksheet(pl_pg)
col_list = list(df.columns)

In [ ]:
# this just makes my life easier

def column_ch(ch):
  val = -1
  for c in list(ch):
    val = (val+1)*26+(ord(c)-65)
  return val

def field(name):
  return(col_list[column_ch(name)])

def fields(start,end=None,skip=1):
  global col_list
  if(end is None):
    return([col_list[column_ch(start)]])
  else:
    return list(col_list[column_ch(start):column_ch(end)+1:skip])



# Define Various Parameters for Picklisting + Clean Data



In [ ]:
# list of teams
team_list = [100,192,253,581,604,668,670,841,972,987,1351,1678,1868,1967,2035,2073,2288,2367,2473,2813,3008,3189,3256,3309,3859,4990,5026,5027,5104,6238,6348,6884,6918,7413,7419,8048,8404,8546,9114,9125,9470,9504]

In [ ]:
auto_cols = fields("J","M")
teleop_cols = fields("R","U")
endgame_cols = fields("W","AC")
yn_cols = fields("H","I")+ fields("N")+fields("W","X") + fields("Z","AA") + fields("AD")
auto_speaker_amp = fields("J","L",2)
tel_speaker_amp = fields("R","T",2)
endgame_yn_cols = fields("X")+fields("AA")
trap_cols = fields("AB","AC")
total_cols = ["auto_total","teleop_total","total_score"]

picklist_cols = [(total_cols[2],"mean"),(field("X"),"mode"),(field("AA"),"mode"),(field("AC"),"mean")]

In [ ]:
for col in auto_cols:
  df[col] =  pd.to_numeric(df[col], errors='coerce', downcast="integer").fillna(0)

for col in teleop_cols:
  df[col] = pd.to_numeric(df[col],errors='coerce', downcast="integer").fillna(0)

for col in trap_cols:
  df[col] = pd.to_numeric(df[col],errors='coerce', downcast="integer").fillna(0)

In [ ]:
# filter out teams that don't exist/not at event
team_num = 'Team Number'
df=df.loc[df[team_num].isin(team_list)]

# makes list of teams
indiv_team = df.groupby(col_list[4])
team_data_list = list(filter(lambda x: isinstance(x,int), list(df[team_num].unique())))

team_data_list.sort()

# Process all the data

In [ ]:
# endgame functions should return most common response - get the modeee :PP
def yn_val(value):
  if value == "Yes":
    return 1
  else:
    return 0

def val_yn(value):
  if value == 1:
    return "Yes"
  else:
    return "No"

In [ ]:
for col in yn_cols:
  df[col] = df.apply({lambda x : yn_val(x[col])},axis=1)

df[yn_cols]

,Preload,💨 Do they leave the start zone?,Does the robot pick up any notes in auto?,🌟 Onstage attempt/success [Attempt],🌟 Onstage attempt/success [Success],🎵 Does the robot harmonize [Attempt],🎵 Does the robot harmonize [Success],🚗 Does the robot park
0,1,0,0,1,1,0,0,0
1,1,1,0,1,1,0,0,0
2,1,1,0,1,1,0,0,0
3,1,0,0,1,1,0,0,1
5,1,1,0,1,1,0,0,0
...,...,...,...,...,...,...,...,...
623,1,1,1,1,1,0,0,0
624,1,0,0,0,0,0,0,0
625,1,0,1,1,0,0,0,0
626,1,1,0,1,0,0,0,1


In [ ]:
df["auto_total"] = df[auto_speaker_amp].sum(axis = 1)
df["teleop_total"] = df[tel_speaker_amp].sum(axis=1)
df["total_score"] = df["auto_total"]+df["teleop_total"]
team_stat_df = indiv_team[auto_cols + teleop_cols + endgame_cols +total_cols ].aggregate(["mean",st.mode]).round()


team_stat_df

<ipython-input-10-2eef557cfa2b>:4: FutureWarning: ['⏰ How long did it take for them to get onstage?'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  team_stat_df = indiv_team[auto_cols + teleop_cols + endgame_cols +total_cols ].aggregate(["mean",st.mode]).round()


🔊 Notes Scored in Auto [Speaker Score]             \
                                              mean       mode   
Team Number                                                     
100                                            0.0   (0.0, 8)   
192                                            0.0   (0.0, 9)   
253                                            0.0  (0.0, 12)   
581                                            2.0   (2.0, 8)   
604                                            3.0   (2.0, 6)   
668                                            1.0  (0.0, 10)   
670                                            0.0  (0.0, 18)   
841                                            0.0  (0.0, 10)   
972                                            0.0  (0.0, 10)   
987                                            2.0   (1.0, 8)   
1351                                           1.0   (1.0, 8)   
1678                                           3.0   (3.0, 8)   
1868                                           0.0  (0.0, 11)   
1967                                           0.0   (0.0, 9)   
2035                                           0.0   (0.0, 9)   
2073                                           1.0  (0.0, 12)   
2288                                           1.0   (0.0, 7)   
2367                                           0.0  (0.0, 12)   
2473                                           1.0   (1.0, 8)   
2813                                           0.0  (0.0, 11)   
3008                                           0.0  (0.0, 12)   
3189                                           0.0   (0.0, 5)   
3256                                           2.0   (1.0, 5)   
3309                                           0.0  (0.0, 17)   
3859                                           0.0   (0.0, 9)   
4990                                           0.0  (0.0, 14)   
5026                                           1.0  (1.0, 10)   
5027                                           0.0  (0.0, 11)   
5104                                           0.0  (0.0, 11)   
6238                                           1.0   (0.0, 7)   
6348                                           0.0  (0.0, 13)   
6884                                           0.0  (0.0, 14)   
6918                                           0.0  (0.0, 10)   
7413                                           0.0  (0.0, 12)   
7419                                           1.0   (1.0, 7)   
8048                                           1.0   (1.0, 9)   
8404                                           2.0   (1.0, 6)   
8546                                           0.0  (0.0, 12)   
9114                                           1.0   (0.0, 7)   
9125                                           1.0   (1.0, 6)   
9470                                           0.0   (0.0, 6)   
9504                                           0.0  (0.0, 12)   

            🔊 Notes Scored in Auto [Speaker Fail]             \
                                             mean       mode   
Team Number                                                    
100                                           0.0   (0.0, 7)   
192                                           0.0   (0.0, 9)   
253                                           0.0  (0.0, 13)   
581                                           0.0  (0.0, 13)   
604                                           1.0   (1.0, 7)   
668                                           0.0  (0.0, 13)   
670                                           1.0   (0.0, 9)   
841                                           0.0  (0.0, 12)   
972                                           1.0   (0.0, 7)   
987                                           0.0  (0.0, 13)   
1351                                          0.0   (0.0, 8)   
1678                                          0.0  (0.0, 16)   
1868                                          0.0  (0.0, 12)   
1967                                          0.0  (0.0, 14)  

In [ ]:
team_stat_df[auto_cols[0]]

,mean,mode
Team Number,,
100,0.0,"(0.0, 8)"
192,0.0,"(0.0, 9)"
253,0.0,"(0.0, 12)"
581,2.0,"(2.0, 8)"
604,3.0,"(2.0, 6)"
668,1.0,"(0.0, 10)"
670,0.0,"(0.0, 18)"
841,0.0,"(0.0, 10)"
972,0.0,"(0.0, 10)"


# Pick out Data

In [ ]:
picklist_data = team_stat_df[picklist_cols]

sort_picklist_data = picklist_data.sort_values(by = [picklist_cols[0]],axis = 0,ascending = False) # write sort code

In [ ]:
picklist = sort_picklist_data.reset_index().values.tolist()

for row in picklist:
    row[2] = val_yn(row[2][0])
    row[3] = val_yn(row[3][0])


# Print picklist onto sheet

In [ ]:
picklist_pg.freeze(0,0)
picklist_pg.update("A2",picklist)
picklist_pg.freeze(1,1)

{'spreadsheetId': '1W-40hZP_zn9xWf4yAHtEEW0jAOLXwpaIt0icdjmVYAw',
 'replies': [{}]}